In [ ]:
import pandas as pd
import numpy as np

: 

In [2]:
!ls

Merged_routes.csv   airports.csv        data_cleaning.ipynb
airlines.csv        airports.dat        routes.csv
airlines.dat        countries.csv       routes.dat


# Read data and comments

In [3]:
airlines = pd.read_csv("airlines.csv", header=None)
airports = pd.read_csv("airports.csv", header=None)
# countries = pd.read_csv("countries.csv", header=None)
routes = pd.read_csv("routes.csv", header=None)

In [4]:
# ID: Unique OpenFlights identifier for this airline.
# Name: Name of the airline.
# Alias: Alias of the airline. For example, All Nippon Airways is commonly known as "ANA".
# IATA: 2-letter IATA code, if available.
# ICAO: 3-letter ICAO code, if available.
# Callsign Airline callsign
# Active: "Y" if the airline is or has until recently been operational
airlines.columns = ["Airline_ID", "Name", "Alias", "IATA", "ICAO", "Callsign", "Country", "Active"]

# DST: Daylight savings time.
# Type: type of the airport. Value "airport" for air terminals, "station" for train stations, "port" for ferry terminals and "unknown" if not known.
airports.columns = ["Airport_ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude", "Altitude", "Timezone", "DST",
 "Tz database time zone", "Type", "Source"]

# It seems countries data is useless
# name	Full name of the country or territory.
# iso_code	Unique two-letter ISO 3166-1 code for the country or territory.
# dafif_code	FIPS country codes as used in DAFIF. Obsolete and primarily of historical interested.
# countries.columns = ["name", "iso_code", "dafif_code"]

# Source airport and Destination airport are represented by IATA or ICAO codes.
# Stops: number of stops on this flight. 0 for direct flight.
routes.columns = ["Airline", "Airline_ID", "Source_airport", "Source_airport_ID", "Destination_airport",
"Destination_airport_ID", "Codeshare", "Stops", "Equipment"]

In [5]:
airlines.head()

,Airline_ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N


In [6]:
airports.head()

,Airport_ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [7]:
# countries.head()

In [8]:
routes.head()

,Airline,Airline_ID,Source_airport,Source_airport_ID,Destination_airport,Destination_airport_ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


# Merge Tables

In [9]:
# Drop useless columns and change data type
airports = airports.loc[:, ["Airport_ID", "City", "Country", "Latitude", "Longitude"]]
airports['Airport_ID'] = airports["Airport_ID"].apply(str)

airlines["Airline_ID"] = airlines['Airline_ID'].apply(str)

In [10]:
# Change the datatype of the columns to be merged on
routes = routes.loc[:, ["Airline_ID", "Source_airport_ID", "Destination_airport_ID", "Stops"]]
routes["Source_airport_ID"] = routes["Source_airport_ID"].apply(str)
routes["Destination_airport_ID"] = routes["Destination_airport_ID"].apply(str)
routes["Airline_ID"] = routes["Airline_ID"].apply(str)

In [11]:
# Get source airport information
routes = routes.merge(airports, left_on="Source_airport_ID", right_on="Airport_ID", how="left").drop(columns=["Airport_ID"])
routes.rename(columns={"City": "Source_City", "Country": "Source_Country", "Latitude": "Source_Latitude",
"Longitude": "Source_Longitude"}, inplace = True)

In [12]:
# Get destination airport information
routes = routes.merge(airports, left_on="Destination_airport_ID", right_on="Airport_ID", how="left").drop(columns=["Airport_ID"])
routes.rename(columns={"City": "Destination_City", "Country": "Destination_Country", "Latitude": "Destination_Latitude",
"Longitude": "Destination_Longitude"}, inplace = True)

In [13]:
# Get airline company information
routes = routes.merge(airlines.loc[:, ["Name", "Active", "Country", "Airline_ID"]], on="Airline_ID", how='left')

In [14]:
# Output csv file
routes.to_csv("Merged_routes.csv")